In [86]:
import kagglehub

dataset_path = kagglehub.dataset_download("saquib7hussain/machine-failure-prediction-dataset")
print("Dataset downloaded to:", dataset_path)


Dataset downloaded to: C:\Users\sachi\.cache\kagglehub\datasets\saquib7hussain\machine-failure-prediction-dataset\versions\1


In [87]:
import os

path = r"C:\Users\sachi\.cache\kagglehub\datasets\saquib7hussain\machine-failure-prediction-dataset\versions\1"
print(os.listdir(path))


['machine_failure_data.csv']


In [88]:
!pip install kagglehub[pandas-datasets]

import kagglehub
from kagglehub import KaggleDatasetAdapter

# Step 1: Set file path based on actual file name you found
file_path = "machine_failure_data.csv"  # change if filename differs

# Step 2: Load dataset as a pandas DataFrame
data = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "saquib7hussain/machine-failure-prediction-dataset",
    file_path
)

print("✅ Dataset loaded successfully!")
print(df.head())



✅ Dataset loaded successfully!
   Temperature    Pressure  Vibration_Level   Humidity  Power_Consumption  \
0    36.475784  360.194000         2.857317  41.952854           5.031080   
1    36.197929  359.684906         2.812744  41.682663           5.046713   
2    36.134769  359.437683         2.731432  41.494057           5.026452   
3    35.803822  360.020996         2.696826  41.175396           5.023470   
4    36.038570  360.580841         2.772655  41.553761           5.034966   

   Failure_Status  
0        0.999786  
1        0.999534  
2        0.999871  
3        0.999934  
4        0.999740  


In [89]:
import numpy as np
import pandas as pd
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [90]:
# Select numerical features to scale
features = ['Temperature', 'Pressure', 'Vibration_Level', 'Humidity', 'Power_Consumption']

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[features])

# Keep Failure_Status as target
occupancy = data['Failure_Status'].values  # This is 0 or 1
scaled_data = np.column_stack([scaled_data, occupancy])

In [91]:
print(data)

        Machine_ID            Timestamp  Temperature  Pressure  \
0      MACHINE_001  2025-01-01 00:00:00        56.23    106.00   
1      MACHINE_002  2025-01-01 00:10:00        36.45    179.39   
2      MACHINE_003  2025-01-01 00:20:00        64.44    432.66   
3      MACHINE_004  2025-01-01 00:30:00        23.28    407.56   
4      MACHINE_005  2025-01-01 00:40:00        27.35    197.15   
...            ...                  ...          ...       ...   
2995  MACHINE_2996  2025-01-21 19:10:00        37.96    138.11   
2996  MACHINE_2997  2025-01-21 19:20:00        54.46    252.42   
2997  MACHINE_2998  2025-01-21 19:30:00        64.83    381.85   
2998  MACHINE_2999  2025-01-21 19:40:00        49.64    420.70   
2999  MACHINE_3000  2025-01-21 19:50:00        73.07    371.52   

      Vibration_Level  Humidity  Power_Consumption  Failure_Status  
0                3.75     77.50              92.06               0  
1                8.02     74.75              30.30               0  


In [92]:
# Define the dimensions
input_dim = 5   # Number of features (Temperature, Pressure, Vibration_Level, Humidity, Power_Consumption)
output_dim = 1  # Target variable (Failure_Status)


In [93]:
print(input_dim)
print(output_dim)

5
1


In [94]:
# GAN Hyperparameters
z_dim = 100 # Latent noise dimension (smaller for structured IoT data)


In [95]:
print(z_dim)

100


In [96]:
import tensorflow as tf
from tensorflow.keras import layers

# Generator Model
def build_generator(z_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=z_dim))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(input_dim + output_dim, activation='sigmoid'))  # Output same shape as input data
    
    print("Generator Model Summary:")
    model.summary(print_fn=lambda x: print(x))  # Forces summary to show in Jupyter
    
    return model

# Call the function to actually print summary
generator = build_generator(z_dim)


Generator Model Summary:


D:\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_49 (Dense)                     │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_50 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_51 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_52 (Dense)                     │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────

In [97]:
import tensorflow as tf
from tensorflow.keras import layers

def build_discriminator(input_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(512, activation='relu', input_dim=input_dim))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output: real(1) or fake(0)

    # Print summary (works in Jupyter)
    print("Discriminator Model Summary:")
    model.summary(print_fn=lambda x: print(x))

    return model

# Call function to build and display model summary
discriminator = build_discriminator(input_dim + output_dim)

Discriminator Model Summary:


Model: "sequential_12"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_54 (Dense)                     │ (None, 512)                 │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_55 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_56 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_57 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────

In [98]:
# Compile the Discriminator
discriminator = build_discriminator(input_dim + output_dim)  # Input is data + Occupancy
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


Discriminator Model Summary:


Model: "sequential_13"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_58 (Dense)                     │ (None, 512)                 │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_59 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_60 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_61 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────

In [99]:
# Compile the GAN model (stacked Generator and Discriminator)
discriminator.trainable = False  # Freeze discriminator during GAN training
generator = build_generator(z_dim)

# GAN Model (stacked generator and discriminator)
z = layers.Input(shape=(z_dim,))
generated_data = generator(z)
validity = discriminator(generated_data)

gan = tf.keras.Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Training the GAN
def train_gan(epochs, batch_size, data):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, data.shape[0], half_batch)
        real_data = data[idx]
        noise = np.random.normal(0, 1, (half_batch, z_dim))
        fake_data = generator.predict(noise)

        # Labels for real and fake data
        real_labels = np.ones((half_batch, 1))
        fake_labels = np.zeros((half_batch, 1))

        # Train on real and fake data
        d_loss_real = discriminator.train_on_batch(real_data, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_data, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_labels)

        # Print the progress
        print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | Acc.: {100 * d_loss[1]}] [G loss: {g_loss}]")

# Train GAN
train_gan(epochs=750, batch_size=64, data=scaled_data)

Generator Model Summary:


Model: "sequential_14"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_62 (Dense)                     │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_63 (Dense)                     │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_64 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_65 (Dense)                     │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────

D:\anaconda\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:86: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0/750 [D loss: 0.6762861013412476 | Acc.: 75.0] [G loss: 0.6405177116394043]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/750 [D loss: 0.6957547068595886 | Acc.: 58.333335876464844] [G loss: 0.6332134008407593]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/750 [D loss: 0.7027938365936279 | Acc.: 55.0] [G loss: 0.6260610222816467]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
3/750 [D loss: 0.7082201242446899 | Acc.: 53.57142639160156] [G loss: 0.6205267906188965]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
4/750 [D loss: 0.7124108076095581 | Acc.: 52.77777862548828] [G loss: 0.6165346503257751]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
5/750 [D loss: 0.7152484059333801 | Acc.: 52.27272415161133] [G loss: 0.6136091351509094]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
6/750 [D loss: 0.7171692848205566 | Acc.: 51.92307662963867] [G loss: 0.6114624738693237]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
7/750 [D loss: 0.7186870574951172 | Acc.: 51.666664123535156] [G loss: 0.6097894906997681]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [100]:
import os
from IPython.display import FileLink

os.makedirs("saved_models", exist_ok=True)

generator.save("saved_models/trained_generator.h5")

print("Generator model saved successfully!")
FileLink("saved_models/trained_generator.h5")

Generator model saved successfully!


D:\case study gan\saved_models\trained_generator.h5

In [101]:
# Number of synthetic data points you want to generate
num_samples = 5000

# Generate random noise (latent vectors)
z_dim = 100  # Dimensionality of the latent space
noise = np.random.normal(0, 1, (num_samples, z_dim))  # Random noise

# Generate synthetic data using the trained generator
synthetic_data = generator.predict(noise)

# Extract features before inverse scaling
synthetic_features = synthetic_data[:, :-1]  # Select all columns except the last (Occupancy)


# If you scaled your data, inverse the scaling for features only
synthetic_features = scaler.inverse_transform(synthetic_features)

# Recombine features and occupancy
synthetic_data = np.column_stack([synthetic_features, synthetic_data[:, -1]])  # Add back Occupancy

# Optionally, separate the features and target
features = synthetic_data[:, :-1]  # Exclude the last column (Occupancy)
failure = synthetic_data[:, -1]  # failure (last column)

# Now, `features` and `failure` can be used as synthetic input data

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [102]:
df=pd.DataFrame(synthetic_data,columns=['Temperature', 'Pressure', 'Vibration_Level', 'Humidity', 'Power_Consumption', 'Failure_Status'])
df

,Temperature,Pressure,Vibration_Level,Humidity,Power_Consumption,Failure_Status
0,79.979767,499.737061,7.017817,89.941200,5.029530,0.999974
1,79.979996,499.739960,7.168107,89.949722,5.020189,1.000000
2,79.979881,499.738770,6.975586,89.945061,5.023594,0.999988
3,79.979225,499.732880,6.906727,89.931084,5.036808,0.999940
4,79.979866,499.738556,7.019904,89.944786,5.024022,0.999986
...,...,...,...,...,...,...
4995,79.979980,499.739685,7.035827,89.948524,5.021396,0.999997
4996,79.978882,499.729095,6.903088,89.926849,5.038849,0.999931
4997,79.979851,499.738342,6.984066,89.944260,5.024932,0.999987
4998,79.979851,499.738312,6.957294,89.944489,5.025774,0.999984


In [103]:
df["Humidity"].value_counts()


Humidity
89.948296    14
89.949310    14
89.949089    13
89.948860    13
89.947174    13
             ..
89.899338     1
89.931633     1
89.943413     1
89.940613     1
89.943436     1
Name: count, Length: 1815, dtype: int64

In [104]:
data

,Machine_ID,Timestamp,Temperature,Pressure,Vibration_Level,Humidity,Power_Consumption,Failure_Status
0,MACHINE_001,2025-01-01 00:00:00,56.23,106.00,3.75,77.50,92.06,0
1,MACHINE_002,2025-01-01 00:10:00,36.45,179.39,8.02,74.75,30.30,0
2,MACHINE_003,2025-01-01 00:20:00,64.44,432.66,4.38,62.55,38.74,0
3,MACHINE_004,2025-01-01 00:30:00,23.28,407.56,4.77,38.96,58.70,0
4,MACHINE_005,2025-01-01 00:40:00,27.35,197.15,8.81,50.49,35.34,1
...,...,...,...,...,...,...,...,...
2995,MACHINE_2996,2025-01-21 19:10:00,37.96,138.11,4.53,53.37,9.86,0
2996,MACHINE_2997,2025-01-21 19:20:00,54.46,252.42,2.13,40.15,75.89,0
2997,MACHINE_2998,2025-01-21 19:30:00,64.83,381.85,2.24,37.36,47.99,0
2998,MACHINE_2999,2025-01-21 19:40:00,49.64,420.70,2.60,82.61,65.41,0


In [105]:
df["Failure_Status"].value_counts()


Failure_Status
0.999999    38
0.999999    38
0.999999    32
0.999998    31
0.999999    30
            ..
0.999927     1
0.999936     1
0.999958     1
0.999959     1
0.999808     1
Name: count, Length: 1055, dtype: int64

In [106]:
import pandas as pd

# ... (your existing code) ...

# Convert failure to a Pandas Series before using value_counts()
failure_series = pd.Series(data['Failure_Status'])
failure_counts = failure_series.value_counts()

print(failure_counts)


Failure_Status
0    2702
1     298
Name: count, dtype: int64
